In [ ]:
# %pip install -U python-dotenv
# %pip install openai

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [3]:
propmt = """
  Estimado chatgpt. El siguiente código es parte de un test e2e en nestjs con jest.

  import { Test, TestingModule } from '@nestjs/testing';
import { INestApplication } from '@nestjs/common';
import * as request from 'supertest';
import { AppModule } from '../../src/app.module';
import { Client } from 'pg';
import {
  Agenda,
  AgendaPostDTO,
} from '../../src/components/agenda/agenda.schema';

describe('ReservationsController (e2e)', () => {
  let app: INestApplication;

  beforeAll(async () => {
    const pg = new Client({
      host: process.env.DB_HOST,
      port: parseInt(process.env.DB_PORT),
      user: process.env.DB_USER,
      password: process.env.DB_PASS,
      database: process.env.DB_NAME,
    });
    await pg.connect();
    await pg.query('CALL seed()');
    await pg.end();
    const moduleFixture: TestingModule = await Test.createTestingModule({
      imports: [AppModule],
    }).compile();

    app = moduleFixture.createNestApplication();

    await app.init();
  });

  afterAll(async () => {
    const pg = new Client({
      host: process.env.DB_HOST,
      port: parseInt(process.env.DB_PORT),
      user: process.env.DB_USER,
      password: process.env.DB_PASS,
      database: process.env.DB_NAME,
    });
    await pg.connect();
    await pg.query('CALL seed()');
    await pg.end();
    await app.close();
  });

  describe('/agenda (POST)', () => {
    it('should return 201 when inserting a valid new agenda', () => {
      const currentDate = new Date('2023-12-01');
      const agendaData: AgendaPostDTO = {
        fecha: currentDate.toISOString(),
        restaurant_theme_id: 1,
      };

      return request(app.getHttpServer())
        .post('/agenda')
        .send(agendaData)
        .expect(201)
        .expect((response) => {
          console.log(response.body);

          const createdAgenda: Agenda = response.body;
          expect(createdAgenda.themeName).toEqual('Restaurante Mexicano');
        });
    });

    it('should return 400 when inserting a reservation on the past', () => {
      const currentDate = new Date('2020-12-01');
      const agendaData: AgendaPostDTO = {
        fecha: currentDate.toISOString(),
        restaurant_theme_id: 1,
      };

      return request(app.getHttpServer())
        .post('/agenda')
        .send(agendaData)
        .expect(400)
        .expect((response) => {
          console.log(response.body);
          const createdReservation = response.body;
          expect(createdReservation.message).toEqual(
            'Bad Request 400 Cant create agenda in the past',
          );
        });
    });

    it('should return 400 when inserting a reservation with bad input', () => {
      const currentDate = new Date('2023-12-02');
      const agendaData: AgendaPostDTO = {
        fecha: currentDate.toISOString(),
        restaurant_theme_id: -1,
      };

      return request(app.getHttpServer())
        .post('/agenda')
        .send(agendaData)
        .expect(400)
        .expect((response) => {
          console.log(response.body);
          const createdReservation = response.body;
          expect(createdReservation.message).toEqual(
            'Bad request: No record inserted - Date is in the past',
          );
        });
    });
  });
});

Resulta que el ultimo test 
it('should return 400 when inserting a reservation with bad input', () => {
      const currentDate = new Date('2023-12-02');
      const agendaData: AgendaPostDTO = {
        fecha: currentDate.toISOString(),
        restaurant_theme_id: -1,
      };

      return request(app.getHttpServer())
        .post('/agenda')
        .send(agendaData)
        .expect(400)
        .expect((response) => {
          console.log(response.body);
          const createdReservation = response.body;
          expect(createdReservation.message).toEqual(
            'Bad request: No record inserted - Date is in the past',
          );
        });
    });

    no esta funcionando correctamente. porque el class-validator no está funcionando en el entorno del test.

    Te muestro el código del main.ts:
    import { NestFactory } from '@nestjs/core';
import { ValidationPipe } from '@nestjs/common';
import { AppModule } from './app.module';

async function bootstrap() {
  const app = await NestFactory.create(AppModule);
  app.useGlobalPipes(new ValidationPipe());
  await app.listen(3000);
}
bootstrap();

Me ayudas a que el useGlobalPipes se ejecute en el entorno de test?

"""

print(get_completion(propmt))

Claro, para que el `useGlobalPipes` se ejecute en el entorno de test, debes agregarlo en la configuración del módulo de testing. Puedes hacerlo de la siguiente manera:

```
import { Test, TestingModule } from '@nestjs/testing';
import { INestApplication } from '@nestjs/common';
import * as request from 'supertest';
import { AppModule } from '../../src/app.module';
import { Client } from 'pg';
import {
  Agenda,
  AgendaPostDTO,
} from '../../src/components/agenda/agenda.schema';

describe('ReservationsController (e2e)', () => {
  let app: INestApplication;

  beforeAll(async () => {
    const pg = new Client({
      host: process.env.DB_HOST,
      port: parseInt(process.env.DB_PORT),
      user: process.env.DB_USER,
      password: process.env.DB_PASS,
      database: process.env.DB_NAME,
    });
    await pg.connect();
    await pg.query('CALL seed()');
    await pg.end();
    const moduleFixture: TestingModule = await Test.createTestingModule({
      imports: [AppModule],
    })
   